In [ ]:
import os
from smolagents import AzureOpenAIServerModel

model = AzureOpenAIServerModel(
    model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
    api_version=os.environ.get("OPENAI_API_VERSION", "")
)

In [ ]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "medical_history").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

for file in (Path.cwd().parent / "data" / "icu_diagnoses").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [ ]:
from medminer.utils.models import DefaultModel
from medminer.task.history import HistoryTask

task = HistoryTask(DefaultModel().model)

for doc in docs:
    task.run(doc.content)